In [ ]:
import cobrakbase
from reframed import *

import cobra
from cobra import Model, Reaction, Metabolite

from math import inf

kbase = cobrakbase.KBaseAPI()

In [ ]:
modelAce = from_cobrapy(kbase.get_from_ws('Acetobacterium_MES1.electrosynthesis.edited', 93204))
media = kbase.get_from_ws('CO2_minimal', 93204)

# constraints
media_const = media.get_media_constraints()
modelAce.compartments.e0.external = True

for rxn_id in modelAce.reactions:
    if 'EX_' in rxn_id:
        rxn = modelAce.reactions[rxn_id]
        rxn.reaction_type = ReactionType.EXCHANGE

for rxn_id in modelAce.get_exchange_reactions():
    rxn = modelAce.reactions[rxn_id]
    if rxn_id[3:] in media_const: 
        lb, ub = media_const[rxn_id[3:]]
        rxn.set_flux_bounds(lb, ub)
    else: 
        rxn.set_flux_bounds(0, 1000) # excretion only

resAce = FBA(modelAce)
print(resAce)

In [ ]:
# Trying to create an individual model of 'Acetobacterium_MES1' from the community model

communityModel = from_cobrapy(kbase.get_from_ws('electro.community.edited.v2', 93204))

modelA = CBModel('modelA')
ext_mets = []

# creating an external compartment
ext_comp_id = 'e0'
comp = Compartment(ext_comp_id, "extracellular environment", external=True)
modelA.add_compartment(comp)

# biomass
biomass_id = 'biomass'
met = Metabolite(biomass_id, "Total biomass", compartment=ext_comp_id)
modelA.add_metabolite(met)

biomass_rxn_id = 'biom'
biomass_rxn = CBReaction(biomass_rxn_id, name="Biomass reaction",
                 reversible=False, stoichiometry={biomass_id: -1},
                 lb=0, ub=inf, objective=1)

modelA.add_reaction(biomass_rxn)

# add internal compartments
new_comp1 = Compartment('c0')
modelA.add_compartment(new_comp1)

# add metabolites        
for m_id, met in communityModel.metabolites.items():
    if m_id[-2:] == 'c1': # internal
        new_met = Metabolite(m_id, compartment='c0')
        new_met.metadata = met.metadata.copy()
        modelA.add_metabolite(new_met)
    elif m_id[-2:] == 'e0' or m_id[-2:] == 'c0': # external
        new_met = Metabolite(m_id, compartment=ext_comp_id)
        new_met.metadata = met.metadata.copy()
        modelA.add_metabolite(new_met)
        ext_mets.append(new_met.id)

# add genes
for g_id, gene in communityModel.genes.items():
    new_id = g_id
    new_gene = Gene(new_id, gene.name)
    new_gene.metadata = gene.metadata.copy()
    modelA.add_gene(new_gene)

# add internal reactions
for r_id, rxn in communityModel.reactions.items():

#     if rxn.reaction_type == ReactionType.EXCHANGE: 
#         continue

    if r_id[-2:] == 'c1' or r_id[-2:] == 'e0' or r_id[-2:] == 'c0' or r_id == 'bio2': 

        new_stoichiometry = {
            m_id if m_id in ext_mets else m_id: coeff
            for m_id, coeff in rxn.stoichiometry.items()
        }

        if r_id == 'bio2':
            new_stoichiometry[biomass_id] = 1

        if rxn.gpr is None:
            new_gpr = None
        else:
            new_gpr = GPRAssociation()
            new_gpr.metadata = rxn.gpr.metadata.copy()

            for protein in rxn.gpr.proteins:
                new_protein = Protein()
                new_protein.genes = [g_id for g_id in protein.genes]
                new_protein.metadata = protein.metadata.copy()
                new_gpr.proteins.append(new_protein)

        new_rxn = CBReaction(
            r_id,
            name=rxn.name,
            reversible=rxn.reversible,
            stoichiometry=new_stoichiometry,
            reaction_type=rxn.reaction_type,
            lb=rxn.lb,
            ub=rxn.ub,
            gpr_association=new_gpr
        )

        modelA.add_reaction(new_rxn)
        new_rxn.metadata = rxn.metadata.copy()

# add exchange reactions
for m_id in ext_mets:
    r_id = f"R_EX_{m_id[2:]}" if m_id.startswith("M_") else f"R_EX_{m_id}"
    rxn = CBReaction(r_id, reversible=True, stoichiometry={m_id: -1}, reaction_type=ReactionType.EXCHANGE)
    modelA.add_reaction(rxn)

resA = FBA(modelA)
print(resA)